<a href="https://colab.research.google.com/github/KanadeSisido/Learning-RecommenderSystems-with-X/blob/main/PyTorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Quick Start
https://docs.pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

### 依存関係のimport

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


### データセットの読み込み

#### datasets.FashionMNISTの引数
- root：取得したデータを保存するパス，dataディレクトリに保存する．
- train：Trueの場合は学習用データをとってくる，Falseの場合はテスト用データを取ってくる．すでに分けられている．
- transform：前処理の設定
- download：データをダウンロードするか．rootで保存したパスに保存する．

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)


### データローダの作成

In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


データローダを観察する

データローダは，ミニバッチの作成などで用いられる．DataLoaderのインスタンス化時に指定したbatch_size個だけ，学習データをピックアップする．呼ばれると（おそらく）ランダムにピックアップされる．

In [ ]:
for X, y in train_dataloader:
  print(X)
  break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [ ]:
print(len(train_dataloader))
print(len(test_dataloader))

938
157


### ニューラルネットワークの作成

#### 概要
ニューラルネットワークの構成は単純で，`nn.Module`を継承したクラスを作成する．

`self.linear_relu_stack`は高階関数`nn.Sequential`が返す関数であることに注意．

`forward`関数内では，入力テンソル`x`を`self.flatten`でフラットなベクトルにし，`self.linear_relu_stack`で順伝播している．

#### 補足説明

`nn.Sequential`内の設定は，以下のコードでは，次のように言える．

入力層`nn.Linear(28 * 28, 512)`　：　28×28の行列をフラットにしたサイズを受取り，512次元で出力する全結合層．活性化関数にReLUを使う．

隠れ層　：　512次元で受取り，512次元で出力する．`nn.Linear(512, 512)`

出力層　：　`nn.Linear(512, 10)`であるが，512次元で受け取って，10クラスで出力する．

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28 * 28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


### 損失関数，オプティマイザの作成

交差エントロピー損失関数，
確率的勾配降下法

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### train関数


`model.train`関数は，モデルが学習中であることをモデルに伝える．これにより，DropOutなどのレイヤを学習に適した挙動に変える．

参照：https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X) # 推論する
    loss = loss_fn(pred, y) # 誤差を取得する

    loss.backward() # 勾配を計算する
    optimizer.step() # パラメータ更新
    optimizer.zero_grad() # 勾配をクリアする

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

### test関数

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device) # バッチサイズだけデータをピックアップして取得
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 要素数10の各クラスの確率×バッチサイズ
  test_loss /= num_batches
  correct /= size

  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

## 学習実行


In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader,model, loss_fn)

Epoch 1
-------------------------------
loss: 2.302627 [   64/60000]
loss: 2.288445 [ 6464/60000]
loss: 2.276178 [12864/60000]
loss: 2.272741 [19264/60000]
loss: 2.261664 [25664/60000]
loss: 2.226735 [32064/60000]
loss: 2.244447 [38464/60000]
loss: 2.207868 [44864/60000]
loss: 2.200129 [51264/60000]
loss: 2.177512 [57664/60000]
Test Error: 
 Accuracy: 34.3%, Avg loss: 2.166847 

Epoch 2
-------------------------------
loss: 2.179189 [   64/60000]
loss: 2.161803 [ 6464/60000]
loss: 2.114512 [12864/60000]
loss: 2.129176 [19264/60000]
loss: 2.091918 [25664/60000]
loss: 2.023762 [32064/60000]
loss: 2.061544 [38464/60000]
loss: 1.982727 [44864/60000]
loss: 1.983951 [51264/60000]
loss: 1.922925 [57664/60000]
Test Error: 
 Accuracy: 49.9%, Avg loss: 1.915762 

Epoch 3
-------------------------------
loss: 1.947694 [   64/60000]
loss: 1.908507 [ 6464/60000]
loss: 1.806904 [12864/60000]
loss: 1.846931 [19264/60000]
loss: 1.753255 [25664/60000]
loss: 1.689466 [32064/60000]
loss: 1.722314 [38464/

## モデルを保存する

In [ ]:
torch.save(model.state_dict(), "model.pth")

## モデルを読み込む

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

## 推論する

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [ ]:
model.eval()

x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
